In [1]:
! pip install pygame

In [2]:
import numpy as np
import random
import math 
import copy
import pygame

pygame 2.1.2 (SDL 2.0.18, Python 3.9.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


# Clases

## Nodos [N]

In [3]:
class Node:
  # propiedades de los nodos [N]
  def __init__(self, tag, x = 0, y = 0, deg = 0):
    self.tag = tag    # etiqueta del nodo
    self.x = x        # índice x
    self.y = y        # índice y
    self.deg = deg    # grado del nodo

  # funciones
  def node_tag(self):
    return self.tag
  def node_deg(self):
    return self.deg
  def node_add_deg(self):
    self.deg = self.deg + 1 

## Aristas [E = (n1, n2)]

In [4]:
class Edge:
  # propiedades de las aristas [E = (n1, n2)]
  def __init__(self, start_node, end_node, weight = 0):
    self.start = start_node   # n1
    self.end = end_node       # n2
    self.weight = weight      # peso de la arista
  
  # funciones
  def edge_n1(self):
    return self.start
  def edge_n2(self):
    return self.end
  def edge_nodes(self):
    return [self.start, self.end]

## Grafos [G(N,E)]

In [5]:
class Graph:
  # propiedades de los grafos [G(N,E)]
  def __init__(self, name, graph_type = 'graph', path = False):
    self.name = name          # nombre del grafo
    self.gtype = graph_type   # tipo del grafo (dirigido o no)
    self.path = path          # camino

    self.nodes = []           # lista de nodos en G
    self.edges = []           # lista de aristas de G
    self.node_lst = []        # lista de nodos en camino
    self.node_weight = []     # lista de pesos de nodos en camino
    self.edge_lst = []        # lista de aristas  en camino

  # funciones
  def add_nodes(self, new_nodes):
    self.nodes.extend(new_nodes)
  def get_nodes(self):
    return self.nodes
  def add_edges(self, new_edges):
    self.edges.extend(new_edges)
  def get_edges(self):
    return self.edges

  ##### función para exportar el grafo #####

  # formato GraphViz (.gv)
  def save_graph(self, graph_name, rand_weight = False):
    
    # generando cadena de texto a guardar
    inside_file = self.gtype + " " + self.name + " {\n"  # nombre y tipo del grafo

    ## agregando nodos ##
    # en camino  
    if self.path:
      for i in range(len(self.nodes)):
        if i in self.node_lst:
          pos_node = self.node_lst.index(i)
          # etiquetado con peso
          inside_file += str(self.nodes[i].tag)+" [label="+str(self.nodes[i].tag)+"("+str(self.node_weight[pos_node])+")];\n"
        # no conectado  
        else: 
          inside_file += str(self.nodes[i].tag)+";\n"
    # sin camino
    else:
      for i in range(len(self.nodes)):
        inside_file += str(self.nodes[i].tag)+";\n"
    
    
    # unión arista
    if self.gtype == 'graph':
      edge_style = "--"
    elif self.gtype == 'digraph':
      edge_style = '->'


    ## agregando aristas ##
    # con peso aleatorio
    if rand_weight:
      for i in range(len(self.edges)):
        inside_file += str(self.edges[i].start) + edge_style + str(self.edges[i].end) + "[w = " +str(round(random.random()*3)+1) + "\"]"+";\n"
    # con peso en camino
    elif self.path:
      for i in range(len(self.edges)):
        inside_file += str(self.edges[i].start) + edge_style + str(self.edges[i].end) + "[w = " +str(self.edges[i].weight) + "\"]"+";\n"
    else: 
      for j in range(len(self.edges)):
        inside_file += str(self.edges[j].start) + edge_style + str(self.edges[j].end) + ";\n"
    # con camino
    if self.path:
      for i in range(len(self.edge_lst)):
        inside_file += str(self.edge_lst[i][0]) + edge_style + str(self.edge_lst[i][1]) +" [color = green,] ;\n"
 
    # final del archivo
    inside_file += "}"

    # escribiendo archivo
    with open(graph_name + ".gv", 'w') as outfile:
      outfile.write(inside_file)

# Modelos de generación de grafos

## Modelo de malla

In [6]:
def Grid(n, m, name, graph_type = 'graph'):
  # listas iniciales
  nodes = []
  edges = []
    
  # generando nodos
  for j in range(m):
    for i in range(n): 
      nodes.append(Node(i, i, j))
   
  # generando aristas # orilla
  for j in range(m-1):
    edges.append(Edge(j*n + n -1, j*n + 2*n -1))
    for i in range(n-1):
      edges.append(Edge(j*n + i, j*n + i + 1)) 
      edges.append(Edge(j*n + i, j*n + i + n))
  for i in range(n-1): # orilla
    edges.append(Edge(n*(m-1) + i, n*(m-1) + i + 1))

  # generando grafo
  new_graph = Graph(name)
  new_graph.add_nodes(nodes)
  new_graph.add_edges(edges)

  return new_graph

In [7]:
#G.append(Grid(3, 10, "Malla30"))
#G.append(Grid(10, 10, "Malla100"))
#G.append(Grid(25, 20, "Malla500"))

## Modelo de Erdös y Rényi

In [8]:
def Erdos_Renyi(n, m, name, graph_type = 'graph', auto=False):
  # iniciales
  matrix = np.zeros((n, n), dtype=int)    
  nodes = []
  edges = []

  # generando n nodos
  for i in range(n):
    nodes.append(Node(i))

  # generando m aristas
  for i in range(m):  
    while True:
      # aristas entre nodos aleatorios    
      start_node = random.randint(0, n - 1)
      end_node = random.randint(0, n - 1)
      if ( (start_node != end_node) or auto ) and matrix[start_node][end_node] == 0:
       break
    matrix[start_node][end_node] = 1
    matrix[end_node][start_node] = 1

    edges.append(Edge(start_node, end_node))
    edges.append(Edge(end_node, start_node))

  # generando grafo            
  new_graph = Graph(name)
  new_graph.add_nodes(nodes)
  new_graph.add_edges(edges)
  
  return new_graph

In [9]:
#G.append(Erdos_Renyi(30, 50, "ErdRen30_50"))
#G.append(Erdos_Renyi(100, 300, "ErdRen100_300"))
#G.append(Erdos_Renyi(500, 1000, "ErdRen500_1000"))

## Modelo de Gilbert

In [10]:
def Gilbert(n, p, name, gtype = 'graph', auto = False):
  # listas iniciales
  nodes = []
  edges = [] 
   
  # generando vértices
  for i in range(n):
    nodes.append(Node(i))

  # generando aristas
  lim = n + auto - 1
  for i in range(n):
    lim=i+auto
    for j in range(lim):
      if random.random() <= p:
        if auto or (not i == j):
          edges.append(Edge(i, j))

  # generando grafo
  new_graph = Graph(name)
  new_graph.add_nodes(nodes)
  new_graph.add_edges(edges)

  return new_graph

In [11]:
#G.append(Gilbert(30, .3, "Gil30_.3"))
#G.append(Gilbert(100, .2, "Gil100_.2"))
#G.append(Gilbert(500, .1, "Gil500_.1"))

## Modelo geográfico simple

In [12]:
# distancia euclideana (entre nodos)
def euclid_dist(x1, y1, x2, y2):
  dist = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
  return dist

def Simple_Geo(n, r, name, graph_type = 'graph', auto = False):   
  # listas iniciales
  nodes = []
  edges = [] 

  x_coords=[]
  y_coords=[]
  
  # generando nodos
  for i in range (n):
    x = random.random()
    x_coords.append(x)
    y = random.random()
    y_coords.append(y)
    nodes.append(Node(i, x = x, y = y))
  
  # generando aristas
  for i in range(n):
    for j in range(n - i - 1 + auto):
      distance = euclid_dist(x_coords[i], y_coords[i], x_coords[n-j-1], y_coords[n-j-1])
      if distance < r:
        edges.append(Edge(i, n-j-1, weight = distance))

  # generando grafo
  new_graph = Graph(name)
  new_graph.add_nodes(nodes)
  new_graph.add_edges(edges)

  return new_graph

In [13]:
#G.append(Simple_Geo(30, .2, "Geo30_.2"))
#G.append(Simple_Geo(100, .1, "Geo100_.1"))
#G.append(Simple_Geo(500, .05, "Geo500_.05"))

## Modelo Barabasi-Albert

In [14]:
def Barabasi_Albert(n, d, name, graph_type = 'graph', auto = False):    
  # listas iniciales
  nodes = []
  edges = [] 
  node_list = []

  # generando nodos
  for i in range(n): 
    nodes.append(Node(i))
    node_list.append(0)
    # comparando con nodos anteriores  
    for j in range (0,i): 
      p = 1 - node_list[j] / d
      # generando aristas
      if random.random() < p:
        edges.append(Edge(i, j))
        node_list[i] += 1
        node_list[j] += 1
        nodes[i].node_add_deg()
        nodes[j].node_add_deg()
        
  # generando grafo
  new_graph = Graph(name)
  new_graph.add_nodes(nodes)
  new_graph.add_edges(edges)

  return new_graph

## Modelo Dorogovtsev-Mendes

In [15]:
def Dorogovtsev_Mendes(n, name, graph_type = 'graph', limit = 0):
  # listas iniciales
  nodes = []
  edges = [] 
    
  #Primeros tres nodos  
  for i in range(3):
    nodes.append(Node(i))
  edges.append(Edge(0,1))
  edges.append(Edge(1,2))
  edges.append(Edge(2,0))

  if limit:
    nodes[0].node_add_deg()
    nodes[1].node_add_deg()
    nodes[2].node_add_deg()
    nodes[0].node_add_deg()
    nodes[1].node_add_deg()
    nodes[2].node_add_deg()

  for i in range(n - 3):
    nodes.append(Node(i + 3))
    if limit:
      flag = True
      while flag:
        pick = random.randint(0, len(edges)-1)
        if nodes[edges[pick].edge_n1()].node_deg() < limit and nodes[edges[pick].edge_n2()].node_deg() < limit:
          flag = False
    else:
      pick = random.randint(0,len(edges)-1)
    
    edges.append(Edge(i+3,edges[pick].edge_n1()))
    edges.append(Edge(i+3,edges[pick].edge_n2()))
    if limit:
      nodes[-1].node_add_deg()
      nodes[-1].node_add_deg()
      nodes[edges[pick].edge_n1()].node_add_deg()
      nodes[edges[pick].edge_n2()].node_add_deg()

  # generando grafo
  new_graph = Graph(name)
  new_graph.add_nodes(nodes)
  new_graph.add_edges(edges)

  return new_graph

In [16]:
#G.append(Dor_Men(30, "DorMen30"))
#G.append(Dor_Men(100, "DorMen100"))
#G.append(Dor_Men(500, "DorMen500"))

# Disposición de grafos P1

In [17]:
def random_start(graph, width, height):
    g = copy.deepcopy(graph)
    for i in range(len(g.nodes)):
        g.nodes[i].x = np.random.randint(0, width)
        g.nodes[i].y =np.random.randint(0, height)
    return g

In [18]:
def r_theta(x1, y1, x2, y2):
  r = math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)
  theta = math.atan2(y2-y1, x2-x1)    
  return r, theta

In [19]:
def vector_sum (f1, a1, f2, a2):
  # componentes
  x = (f1 * math.cos(a1)) + (f2 * math.cos(a2))
  y = (f1 * math.sin(a1)) + (f2 * math.sin(a2))
  r, theta = r_theta(x, y, 0, 0)
  return r, theta

## Resorte

In [20]:
def spring(graph, c1, c2, c3, c4, width, height):
  g = copy.deepcopy(graph)
  forces = [(0,0)] * len(g.nodes)
    
  # F de atracción
  for i in range(len(g.edges)):
    n1 = g.edges[i].start
    n2 = g.edges[i].end

    coord_1 = (g.nodes[n1].x, g.nodes[n1].y)
    coord_2 = (g.nodes[n2].x, g.nodes[n2].y)

    size, angle = r_theta(coord_1[0], coord_1[1], coord_2[0], coord_2[1])
    if size > 0:
      F_a = c1 * np.log(size / c2)
    else:
      F_a = 0
    
    force_a, angle_a = vector_sum(F_a, angle + math.pi, forces[n1][0], forces[n1][1])
    forces[n1] = (force_a, angle_a)
    force_a, angle_a = vector_sum(F_a, angle + math.pi, forces[n2][0], forces[n2][1])
    forces[n2] = (force_a, angle_a)
  
  # F de repulsión
  for i in range(len(g.nodes)):
    for j in range(len(g.nodes)):
      if i != j:
        coord_1 = (g.nodes[n1].x, g.nodes[n1].y)
        coord_2 = (g.nodes[n2].x, g.nodes[n2].y)
        size, angle = r_theta(coord_1[0], coord_1[1], coord_2[0], coord_2[1])
        #fuerza
        if size > 0:
          F_r = c3 / math.sqrt(size)
        else:
          F_r = c3 * 5
       
        force_r, angle_r = vector_sum(F_r, angle, forces[i][0], forces[i][1])
        forces[i] = (force_r, angle_r)
  
  # Gran atractor
  atractor_x = width / 2
  atractor_y = height / 2

  for i in range(len(g.nodes)):
    coord_1 = (g.nodes[n1].x, g.nodes[n1].y)
    size, angle = r_theta(coord_1[0], coord_1[1], atractor_x, atractor_y)
    if size > 0:
      FA = size * 0
    else:
      FA = 0
    force_atr, angle_atr = vector_sum(FA, angle + math.pi, forces[i][0], forces[i][1])
    forces[i] = (force_atr, angle_atr)
  
  # actualizando posiciones
  for i in range(len(g.nodes)):
    x = c4 * forces[i][0] * math.cos(forces[i][1])
    y = c4 * forces[i][0] * math.sin(forces[i][1])
   
    g.nodes[i].x = g.nodes[i].x + x
    g.nodes[i].y = g.nodes[i].y + y
    # ajuste dimensional
    if g.nodes[i].x >= width:
      g.nodes[i].x = width-1
    elif g.nodes[i].x < 1:
      g.nodes[i].x = 1

    if g.nodes[i].y >= height:
      g.nodes[i].y = height - 1
    elif g.nodes[i].y < 1:
      g.nodes[i].y = 1 

  return g

# Disposición de Grafos P2

## Force-directed: Barnes y Hut

In [21]:
class quadtree:
  def __init__(self, x_boundary, y_boundary):
    self.center = ((x_boundary[0] + x_boundary[1]) / 2, 
                   (y_boundary[0] + y_boundary[1]) / 2)
    self.center_mass = 0
    self.mass = 0
    self.quad = [[], [], [], []]
    self.x_boundary = x_boundary
    self.y_boundary = y_boundary
    self.nodes = []

  def get_center_of_mass(self):
    sum_x = 0
    sum_y = 0
    for i in range(len(self.nodes)):
      sum_x += self.nodes[i][0]
      sum_y += self.nodes[i][1]

    if self.mass <= 0:
      self.center_mass = self.center
    if self.mass > 0:
      self.center_mass = (sum_x / self.mass, sum_y / self.mass)

  def add_mass(self):
    self.mass += 1

In [22]:
# Genera particiones
def quadrants(og_quad, capacity):
  og_quad.quad[0] = quadtree((og_quad.x_boundary[0], og_quad.center[0]), 
                               (og_quad.y_boundary[0], og_quad.center[1]))  # north-west
  og_quad.quad[1] = quadtree((og_quad.center[0], og_quad.x_boundary[1]), 
                               (og_quad.y_boundary[0], og_quad.center[1]))  # north-east
  og_quad.quad[2] = quadtree((og_quad.x_boundary[0], og_quad.center[0]), 
                               (og_quad.center[1], og_quad.y_boundary[1]))  # south-west
  og_quad.quad[3] = quadtree((og_quad.center[0], og_quad.x_boundary[1]), 
                               (og_quad.center[1], og_quad.y_boundary[1]))  # south-east

  for i in range(len(og_quad.nodes)):
    if og_quad.nodes[i][0] < og_quad.center[0]:  
      # NE
      if og_quad.nodes[i][1] > og_quad.center[1]:  
        og_quad.quad[1].add_mass()
        og_quad.quad[1].nodes.append(og_quad.nodes[i])
      # SE
      else:        
        og_quad.quad[3].add_mass()
        og_quad.quad[3].nodes.append(og_quad.nodes[i])
    if og_quad.nodes[i][0] < og_quad.center[0]:  
      # NW
      if og_quad.nodes[i][1] > og_quad.center[1]: 
        og_quad.quad[0].add_mass()
        og_quad.quad[0].nodes.append(og_quad.nodes[i])
      # SW
      else:           
        og_quad.quad[2].add_mass()
        og_quad.quad[2].nodes.append(og_quad.nodes[i])            
    
  for i in range(4):
    # nuevos centros de masa
    og_quad.quad[i].get_center_of_mass()

    # subdivisión
    if og_quad.quad[i].mass > capacity:
      og_quad.quad[i] = quadrants(og_quad.quad[i], capacity)
    
  return og_quad

## Fruchterman y Reigold

In [23]:
def zoom(quad, coords, force, max_alpha, cte):
  new_coords = (quad.center_mass[0], quad.center_mass[1])
  r, theta = r_theta(coords[0], coords[1], new_coords[0], new_coords[1])
  
  if quad.mass == 0:
    return force
  if r > 0:
    alpha = (quad.x_boundary[1] - quad.x_boundary[0]) / r
  else:
    return force

  # sub dividido
  if alpha <= max_alpha:
    for i in range(4):
      if quad.quadtree[i] != []:
        force = zoom(quad.quadtree[i], coords, force, max_alpha, cte)
  # suficiente
  if alpha < max_alpha:
    F = quad.mass * cte**2 / r
    f, angle = vector_sum(F, theta, force[0], force[1])
    force = (f, angle)
  return force

In [24]:
def FruRei_quad(graph, capacity, temp, cool, width, height, max_theta):
  quad = quadtree((0, width), (0, height))  # espacio total
  forces = [(0,0) * len(graph.nodes)]
  temp = temp * cool
  # m = 1
  for i in range(len(graph.nodes)):
    quad.nodes.append([graph.nodes[i].x, graph.nodes[i].y])
    quad.add_mass()
  quad.get_center_of_mass()
  # particiones
  if quad.mass > capacity:
    quad = quadrants(quad, capacity)
    
  # F atracción
  for j in range(len(graph.edges)):
    n1 = graph.edges[i].start
    n2 = graph.edges[i].end
    coords_1 = (graph.nodes[n1].x, graph.nodes[n1].y)
    coords_2 = (graph.nodes[n2].x, graph.nodes[n2].y)
    r, theta = r_theta(coords_1[0], coords_1[1], coords_2[0], coords_2[1])
    
    if r > 0:
      F_a = (r ** 2) / k
    else:
      F_a = 0
    f, angle = vector_sum(F_a, theta + math.pi, forces[n1][0], forces[n1][1])
    forces[n1] = (f, angle)
    f, angle = vector_sum(F_a, theta + math.pi, forces[n2][0], forces[n2][1])
    forces[n2] = (f, angle)

  # F repulsión
  for k in range(len(graph.nodes)):
    F_r = (0,0)
    coords = (graph.nodes[k].x, graph.nodes[k].y)
    F_r = zoom(quad, coords, F_r, max_alpha, cte)
    forces[i] = F_r

  #posiciones nuevas
  for i in range(len(graph.nodes)):
    x = min(forces[i][0], temp) * math.cos(forces[i][1])
    y = min(forces[i][0], temp) * math.sin(forces[i][1])
    graph.nodes[i].x = graph.nodes[i].x + x
    graph.nodes[i].y = graph.nodes[i].y + y

    # horizontal
    if graph.nodes[i].x < 1:
      graph.nodes[i].x = random.randint(1,3)
    elif graph.nodes[i].x >= width:
      graph.nodes[i].x = width - random.randint(1,3)
    # vertical
    if graph.nodes[i].y < 1:
      graph.nodes[i].y = random.randint(1,3)
    elif graph.nodes[i].y >= height:
      graph.nodes[i].y = height - random.randint(1,3)

  return graph, temp

## Pygame

In [25]:
class App:
  # formato (pantalla, colores, texto)
  def __init__(self, graph):
    self._running = True
    self._display_surf = None
    self.width = 1900
    self.height = 1000
    self.size = self.width, self.height
    self.graph = graph
  
  def on_init(self):
    pygame.init()
    self._display_surf = pygame.display.set_mode(self.size) # pygame.HWSURFACE | pygame.DOUBLEBUF)
    self._running = True 

    # colores
    background_color = (224, 224, 224)
    text_color = (32, 32, 32)
    text_width = int(self.width * .7)

    self.text_color = text_color
    self.text_width = text_width
    self.background = background_color
    self._display_surf.fill(background_color)

    # Texto
    self.title_font = pygame.font.Font('C:\\Windows\\Fonts\\Bahnschrift.ttf', 15)
    self.normal_font = pygame.font.Font('C:\\Windows\\Fonts\\Bahnschrift.ttf', 11)

    text = self.title_font.render('Datos del grafo', True, text_color)
    self._display_surf.blit(text, (text_width, 40))
    text = self.normal_font.render('Nodos', True, text_color)
    self._display_surf.blit(text, (text_width, 75))
    text = self.normal_font.render('Aristas', True, text_color)
    self._display_surf.blit(text, (text_width, 100))

    tex = str(len(self.graph.nodes))
    text = self.normal_font.render(tex, True, text_color, background_color)
    self._display_surf.blit(text, (text_width + 180, 75))

    tex = str(len(self.graph.edges))
    text = self.normal_font.render(tex, True, text_color, background_color)
    self._display_surf.blit(text, (text_width + 180, 75))
    
    text = self.normal_font.render('C1', True, text_color)
    self._display_surf.blit(text, (text_width, 125))
    text = self.normal_font.render('C2', True, text_color)
    self._display_surf.blit(text, (text_width, 145))
    text = self.normal_font.render('C3', True, text_color)
    self._display_surf.blit(text, (text_width, 165))
    text = self.normal_font.render('C4', True, text_color)
    self._display_surf.blit(text, (text_width, 185))

    self.c1 = 1
    self.c2 = 1
    self.c3 = 1
    self.c4 = 1

    self.capacity = 1
    self.theta = 0.25

    self.temp = 15
    self.cool = 0.7
    self.cte = 0.4

    text = self.normal_font.render('Barnes y Hut', True, text_color)
    self._display_surf.blit(text, (text_width, 210))
    text = self.normal_font.render('capacidad', True, text_color)
    self._display_surf.blit(text, (text_width, 230))
    text = self.normal_font.render('theta', True, text_color)
    self._display_surf.blit(text, (text_width, 250))
    
    text = self.normal_font.render('Fruchterman y Reigold', True, text_color)
    self._display_surf.blit(text, (text_width, 300))
    text = self.normal_font.render('temp', True, text_color)
    self._display_surf.blit(text, (text_width, 320))
    text = self.normal_font.render('cool', True, text_color)
    self._display_surf.blit(text, (text_width, 340))
    text = self.normal_font.render('Cte', True, text_color)
    self._display_surf.blit(text, (text_width, 360))

    # color
    self.inactive_color = pygame.Color('azure4')
    self.active_color = pygame.Color('coral4')

    self.c1_color = self.inactive_color
    self.c2_color = self.inactive_color
    self.c3_color = self.inactive_color
    self.c4_color = self.inactive_color

    self.quad_color = self.inactive_color
    self.capacity_color = self.inactive_color
    self.theta_color = self.inactive_color

    self.cte_color = self.inactive_color
    self.temp_color = self.inactive_color
    self.cool_color = self.inactive_color
    self.FR_color = self.inactive_color
    

    self.c1_active = False
    self.c2_active = False
    self.c3_active = False
    self.c4_active = False

    self.spring_on = False
    # barnes y hut
    self.quad_on = False
    self.capacity_on = False
    self.theta_on = False
    # fruchterman y reigold
    self.cte_on = False
    self.temp_on = False
    self.cool_on = False
    self.FR_on = True
    self.c_color = self.inactive_color

    self.c1_in = pygame.Rect(text_width, 135, 120, 20)
    self.c1_text = '1'
    pygame.draw.rect(self._display_surf, self.c1_color, self.c1_in, width = 2)
    
    self.c2_in = pygame.Rect(text_width, 155, 120, 20)
    self.c2_text = '1'
    pygame.draw.rect(self._display_surf, self.c2_color, self.c2_in, width = 2)
    
    self.c3_in = pygame.Rect(text_width, 175, 120, 20)
    self.c3_text = '1'
    pygame.draw.rect(self._display_surf, self.c3_color, self.c3_in, width = 2)
    
    self.c4_in = pygame.Rect(text_width, 200, 120, 20)
    self.c4_text = '1'
    pygame.draw.rect(self._display_surf, self.cte_color, self.c4_in, width = 2)

    self.cte_in = pygame.Rect(text_width, 250, 120, 20)
    self.cte_text = '.4'
    pygame.draw.rect(self._display_surf, self.cte_color, self.cte_in, width = 2)
    
    self.temp_in = pygame.Rect(text_width, 270, 120, 20)
    self.temp_text = '100'
    pygame.draw.rect(self._display_surf, self.temp_color, self.temp_in, width = 2)
    
    self.cool_in = pygame.Rect(text_width, 290, 120, 20)
    self.cool_text = '.7'
    pygame.draw.rect(self._display_surf, self.cool_color, self.cool_in, width = 2)

    self.capacity_in = pygame.Rect(text_width, 350, 120, 20)
    self.capacity_text = '1'
    pygame.draw.rect(self._display_surf, self.capacity_color, self.capacity_in, width = 2)
    
    self.theta_in = pygame.Rect(text_width, 370, 120, 20)
    self.theta_text = '.25'
    pygame.draw.rect(self._display_surf, self.theta_color, self.theta_in, width = 2)

    self.quad_in = pygame.Rect(250, text_width, 80, 20)
    pygame.draw.rect(self._display_surf, self.quad_color, self.quad_in, width=2)
    self.FR_in = pygame.Rect(250, text_width, 200, 20)
    pygame.draw.rect(self._display_surf, self.FR_color, self.FR_in, width=2)
        

   
    # nodos inicialmente en aleatorio
    self.graph = random_start(self.graph, self.width * .8, self.height * .8)
  
  # cierra
  def on_event(self, event):
    if event.type == pygame.QUIT:
      self._running = False


  ##### loop #####
  def loop(self):
    #pygame.time.delay(2)
    surface = pygame.Surface((30, 30))
    surface = pygame.transform.scale(surface, (int(self.width * .7), int(self.height * .7)))

    #if spring_on:
     # self.graph = spring(self.graph, self.c1, self.c2, self.c3, self.c4, self.width * .7, self.height * .7)
    if self.quad_on:
      self.graph = quadrants(self.graph, self.capacity)
    elif self.FR_on:
      self.graph = FruRei_quad(self.graph, self.cte, self.temp, self.cool, self.width*.7, self.height*.7, self.theta)
    
    
    # dibujando nodos
    for i in range(len(self.graph.nodes)):
      pygame.draw.circle(surface, (102, 102, 255), (self.graph.nodes[i].x, self.graph.nodes[i].y), 3)

    # dibujando aristas
    for j in range(len(self.graph.edges)):
      n1 = self.graph.edges[j].start
      n2 = self.graph.edges[j].end
      coord_1 = (self.graph.nodes[n1].x, self.graph.nodes[n1].y)
      coord_2 = (self.graph.nodes[n2].x, self.graph.nodes[n2].y)
      pygame.draw.line(surface, (204, 204, 255), coord_1, coord_2)

    self._display_surf.blit(surface, (55, 55))

    text = self.normal_font.render(self.c1_text, True, self.text_color, pygame.Color('slateblue4'))
    self._display_surf.blit(text, (self.text_width, 130))
    pygame.draw.rect(self._display_surf, self.c1_color, self.c1_in, width = 2)

    text = self.normal_font.render(self.c2_text, True, self.text_color, pygame.Color('slateblue4'))
    self._display_surf.blit(text, (self.text_width, 150))
    pygame.draw.rect(self._display_surf, self.c2_color, self.c2_in, width = 2)

    text = self.normal_font.render(self.c3_text, True, self.text_color, pygame.Color('slateblue4'))
    self._display_surf.blit(text, (self.text_width, 170))
    pygame.draw.rect(self._display_surf, self.c3_color, self.c3_in, width = 2)

    text = self.normal_font.render(self.c4_text, True, self.text_color, pygame.Color('slateblue4'))
    self._display_surf.blit(text, (self.text_width, 190))
    pygame.draw.rect(self._display_surf, self.c4_color, self.c4_in, width = 2)

    pass
  
  def on_render(self):
    pygame.display.flip()
    pass

  def on_cleanup(self):
    pygame.quit()

  def on_execute(self):
    if self.on_init() == False:
      self._running = False
    while( self._running):
      for event in pygame.event.get():
        self.on_event(event)
      self.loop()
      self.on_render()
    self.on_cleanup()

In [ ]:
#G = Grid(10, 10, "Malla100")
#G = Grid(25, 20, "Malla500")

#G = Erdos_Renyi(100, 300, "ErdRen100")
#G = Erdos_Renyi(500, 1000, "ErdRen500")

G = Gilbert(100, .1, "Gil100")
#G = Gilbert(500, .03, "Gil500")

#G = Simple_Geo(100, .1, "Geo100")
#G = Simple_Geo(500, .05, "Geo500")

#G = Barabasi_Albert(100, 5, "BarAlb100")
#G = Barabasi_Albert(500, 3, "BarAlb500")

#G = Dorogovtsev_Mendes(100,"DorMen100")
#G = Dorogovtsev_Mendes(500,"DorMen500")

App(G)
App(G).on_execute()